In [1]:
from . import filter_talon_transcripts as filt
from .. import dstruct as dstruct
from .. import length_utils as lu
from . import post_utils as putils
from .. import query_utils as qutils
from .. import talon as talon

ImportError: cannot import name 'filter_talon_transcripts' from '__main__' (unknown location)

In [187]:
import numpy as np

In [45]:
# TALON: Techonology-Agnostic Long Read Analysis Pipeline
# Author: Dana Wyman
# -----------------------------------------------------------------------------
# Queries for interacting with a TALON database

import sqlite3

# TALON: Techonology-Agnostic Long Read Analysis Pipeline
# Author: Dana Wyman
# -----------------------------------------------------------------------------
# Queries for working with exon and transcript lengths

def get_all_exon_lengths(cursor, build):
    """ Compute all exon lengths and store in a dict """

    exon_lengths = {}
    cursor.execute(""" SELECT edge_ID,
                          loc1.position AS pos1,
                          loc2.position AS pos2,
                          abs(loc1.position - loc2.position) + 1 AS diff
                       FROM edge 
                       LEFT JOIN location AS loc1 ON edge.v1 = loc1.location_ID
                       LEFT JOIN location AS loc2 ON edge.v2 = loc2.location_ID
                       WHERE edge_type = 'exon' 
                       AND loc1.genome_build = '%s'
                       AND loc2.genome_build = '%s' """ % (build, build))

    for exon in cursor.fetchall():
        exon_ID = exon['edge_ID']
        length = exon['diff']
        exon_lengths[exon_ID] = length

    return exon_lengths

def get_transcript_length(transcript_row, exon_lengths):
    """ Compute the length of the supplied transcript model based on its
        exons. Expected input format consists of a transcript row from a
        TALON database. """

    length = 0
    start_exon = transcript_row['start_exon']
    end_exon = transcript_row['end_exon']
    n_exons = transcript_row['n_exons']

    if n_exons == 1:
        return exon_lengths[start_exon]
    else:
        jn_path =  transcript_row['jn_path'].split(",")
        all_exons = [start_exon] + [int(x) for x in jn_path[1::2]] + [end_exon]
       
        for exon in all_exons:
            length += exon_lengths[exon]

        return length


    


def fetch_reproducible_intergenic(cursor, datasets):
    """ Return the gene and transcript ID of any intergenic transcripts that were
        found in at least two of the supplied datasets """

    datasets = format_for_IN(datasets)
    query = """SELECT gene_ID,
                      a.transcript_ID
               FROM abundance as a
               LEFT JOIN transcript_annotations as ta
                   ON ta.ID = a.transcript_ID
               LEFT JOIN transcripts
                   ON transcripts.transcript_ID = a.transcript_ID
               WHERE ta.attribute = 'intergenic_transcript'
               AND a.dataset IN """ + datasets + \
           """ GROUP BY a.transcript_ID
               HAVING count(*) > 1;"""

    cursor.execute(query)
    intergenic = [(x[0], x[1], "intergenic_transcript") for x in cursor.fetchall()]
    return intergenic

def fetch_reproducible_antisense(cursor, datasets):
    """ Return the gene and transcript ID of any antisense transcripts that were
        found in at least two of the supplied datasets """

    datasets = format_for_IN(datasets)
    query = """SELECT gene_ID,
                      a.transcript_ID
               FROM abundance as a
               LEFT JOIN transcript_annotations as ta
                   ON ta.ID = a.transcript_ID
               LEFT JOIN transcripts
                   ON transcripts.transcript_ID = a.transcript_ID
               WHERE ta.attribute = 'antisense_transcript'
               AND a.dataset IN """ + datasets + \
           """ GROUP BY a.transcript_ID
               HAVING count(*) > 1;"""

    cursor.execute(query)
    antisense = [(x[0], x[1], "antisense_transcript") for x in cursor.fetchall()]
    return antisense

def fetch_reproducible_NNCs(cursor, datasets):
    """ Return the gene and transcript ID of any NNC transcripts that were
        found in at least two of the supplied datasets """

    datasets = format_for_IN(datasets)
    query = """SELECT gene_ID, 
                      a.transcript_ID 
               FROM abundance as a
	       LEFT JOIN transcript_annotations as ta
	           ON ta.ID = a.transcript_ID
               LEFT JOIN transcripts
	           ON transcripts.transcript_ID = a.transcript_ID
	       WHERE ta.attribute = 'NNC_transcript'
	       AND a.dataset IN """ + datasets + \
           """ GROUP BY a.transcript_ID
               HAVING count(*) > 1;"""

    cursor.execute(query)
    NNC = [(x[0], x[1], "NNC_transcript") for x in cursor.fetchall()]
    return NNC

def fetch_reproducible_NICs(cursor, datasets):
    """ Return the gene and transcript ID of any NIC transcripts that were
        found in at least two of the supplied datasets """

    datasets = format_for_IN(datasets)
    query = """SELECT gene_ID,
                      a.transcript_ID
               FROM abundance as a
               LEFT JOIN transcript_annotations as ta
                   ON ta.ID = a.transcript_ID
               LEFT JOIN transcripts
                   ON transcripts.transcript_ID = a.transcript_ID
               WHERE ta.attribute = 'NIC_transcript'
               AND a.dataset IN """ + datasets + \
           """ GROUP BY a.transcript_ID
               HAVING count(*) > 1;"""

    cursor.execute(query)
    NIC = [(x[0], x[1], "NIC_transcript") for x in cursor.fetchall()]
    return NIC

def fetch_reproducible_ISMs(cursor, datasets):
    """ Return the gene and transcript ID of any ISM transcripts that were
        found in at least two of the supplied datasets """

    datasets = format_for_IN(datasets)
    transcripts_seen = {}

    # To label novelty, perform queries separately for suffix, prefix, and
    # regular ISMs
    query = """SELECT gene_ID,
                      a.transcript_ID
               FROM abundance as a
               LEFT JOIN transcript_annotations as ta
                   ON ta.ID = a.transcript_ID
               LEFT JOIN transcripts
                   ON transcripts.transcript_ID = a.transcript_ID
               WHERE ta.attribute = 'ISM-prefix_transcript'
               AND a.dataset IN """ + datasets + \
           """ GROUP BY a.transcript_ID
               HAVING count(*) > 1;"""

    cursor.execute(query)
    ISMs = [(x[0], x[1], "ISM-prefix_transcript") for x in cursor.fetchall()]

    for entry in ISMs:
        transcripts_seen[entry[1]] = 1

    query = """SELECT gene_ID,
                      a.transcript_ID
               FROM abundance as a
               LEFT JOIN transcript_annotations as ta
                   ON ta.ID = a.transcript_ID
               LEFT JOIN transcripts
                   ON transcripts.transcript_ID = a.transcript_ID
               WHERE ta.attribute = 'ISM-suffix_transcript'
               AND a.dataset IN """ + datasets + \
           """ GROUP BY a.transcript_ID
               HAVING count(*) > 1;"""

    cursor.execute(query)
    suffix_ISMs = [(x[0], x[1], "ISM-suffix_transcript") for x in cursor.fetchall()]
    # Only add suffix ISM transcript if it isn't already on the list
    for entry in suffix_ISMs:
        if entry[1] not in transcripts_seen:
            ISMs.append(entry)
            transcripts_seen[entry[1]] = 1

    query = """SELECT gene_ID,
                      a.transcript_ID
               FROM abundance as a
               LEFT JOIN transcript_annotations as ta
                   ON ta.ID = a.transcript_ID
               LEFT JOIN transcripts
                   ON transcripts.transcript_ID = a.transcript_ID
               WHERE ta.attribute = 'ISM_transcript'
               AND a.dataset IN """ + datasets + \
           """ GROUP BY a.transcript_ID
               HAVING count(*) > 1;"""

    cursor.execute(query)
    all_ISMs = [(x[0], x[1], "other_ISM_transcript") for x in cursor.fetchall()]
    # Only add ISM transcript if it isn't already on the list
    for entry in all_ISMs:
        if entry[1] not in transcripts_seen:
            ISMs.append(entry)
            transcripts_seen[entry[1]] = 1

    return ISMs

def fetch_known_transcripts_with_gene_label(cursor, datasets):
    """ Fetch known transcripts along with the gene they belong to """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT gene_ID,transcript_ID FROM observed
                   LEFT JOIN transcript_annotations AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'transcript_status' AND ta.value = 'KNOWN')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    known_transcripts = [(x[0], x[1], "FSM_transcript") for x in cursor.fetchall()]
    return known_transcripts

def fetch_NIC_transcripts_with_gene_label(cursor, datasets):
    """ Fetch NIC transcripts along with the gene they belong to """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT gene_ID,transcript_ID FROM observed
                   LEFT JOIN transcript_annotations AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'NIC_transcript')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    known_transcripts = [(x[0], x[1], "NIC_transcript") for x in cursor.fetchall()]
    return known_transcripts

def count_observed_reads(cursor, datasets):
    """ Count the number of observed reads for the provided datasets """

    datasets = format_for_IN(datasets)
    query = "SELECT COUNT(obs_ID) FROM observed WHERE dataset IN " + datasets
    cursor.execute(query)
    reads = cursor.fetchone()[0]
    return reads

def fetch_all_known_genes_detected(cursor, datasets):
    """ Get the IDs of all known genes found in a particular dataset (no 
        filter with respect to the type of transcript detected). """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(gene_ID) FROM observed
                   LEFT JOIN gene_annotations AS ga ON ga.ID = observed.gene_ID
                   WHERE (ga.attribute = 'gene_status' AND ga.value = 'KNOWN')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    known_genes = [x[0] for x in cursor.fetchall()]
    return known_genes

def count_known_genes_detected(cursor, dataset):
    """ Count the number of known genes detected in the dataset (no filter
        with respect to the type of transcript detected). """

    known_genes = fetch_all_known_genes_detected(cursor, dataset)
    return len(known_genes)

def count_novel_genes_detected(cursor, dataset):
    """ Count the number of novel genes detected in the dataset (no filter
        with respect to the type of transcript detected). """

    novel_genes = fetch_all_novel_genes_detected(cursor, dataset)
    return len(novel_genes)

def fetch_all_novel_genes_detected(cursor, datasets):
    """ Get the IDs of all novel genes found in a particular dataset (no
        filter with respect to the type of transcript detected). """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(gene_ID) FROM observed
                   LEFT JOIN gene_annotations AS ga ON ga.ID = observed.gene_ID
                   WHERE (ga.attribute = 'gene_status' AND ga.value = 'NOVEL')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    novel_genes = [x[0] for x in cursor.fetchall()]
    return novel_genes

def fetch_all_known_transcripts_detected(cursor, datasets):
    """ Get the IDs of all transcripts annotated as known. Does not include 
        novel FSMs """
 
    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(transcript_ID) FROM observed
                   LEFT JOIN transcript_annotations AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'transcript_status' AND ta.value = 'KNOWN')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    known_transcripts = [x[0] for x in cursor.fetchall()]
    return known_transcripts

def fetch_FSM_novel_transcripts(cursor, dataset):
    """ Fetch IDs of novel FSMs observed in the current dataset """

    query = """SELECT DISTINCT(transcript_ID) FROM observed
                   LEFT JOIN transcript_annotations AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'FSM_transcript' AND ta.value = 'TRUE')
                   AND observed.dataset = ?;"""
    cursor.execute(query, [dataset])
    FSM_transcripts = [x[0] for x in cursor.fetchall()]
    return FSM_transcripts

def fetch_novel_transcripts(cursor, datasets):
    """ Fetch IDs of novel transcripts observed in the current dataset """
    datasets = format_for_IN(datasets)

    query = """SELECT DISTINCT(transcript_ID) FROM observed
                   LEFT JOIN transcript_annotations AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'transcript_status' AND ta.value = 'NOVEL')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    transcripts = [x[0] for x in cursor.fetchall()]
    return transcripts

def fetch_antisense_genes(cursor, datasets):
    """ Fetch IDs of antisense genes observed in the dataset(s) """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(gene_ID) FROM observed
                   LEFT JOIN gene_annotations AS ga ON ga.ID = observed.gene_ID
                   WHERE (ga.attribute = 'antisense_gene')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    genes = [x[0] for x in cursor.fetchall()]
    return genes

def fetch_intergenic_novel_genes(cursor, datasets):
    """ Fetch IDs of novel genes denoted as intergenic """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(gene_ID) FROM observed
                   LEFT JOIN gene_annotations AS ga ON ga.ID = observed.gene_ID
                   WHERE (ga.attribute = 'intergenic_novel')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    genes = [x[0] for x in cursor.fetchall()]
    return genes

def fetch_all_ISM_transcripts(cursor, datasets):
    """ Fetch IDs of all ISM transcripts """
    
    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(transcript_ID) FROM observed
                   LEFT JOIN transcript_annotations 
                       AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'ISM_transcript')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    transcripts = [x[0] for x in cursor.fetchall()]
    return transcripts

def fetch_prefix_ISM_transcripts(cursor, datasets):
    """ Fetch IDs of all ISM prefix transcripts """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(transcript_ID) FROM observed
                   LEFT JOIN transcript_annotations
                       AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'ISM-prefix_transcript')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    transcripts = [x[0] for x in cursor.fetchall()]
    return transcripts

def fetch_suffix_ISM_transcripts(cursor, datasets):
    """ Fetch IDs of all ISM suffix transcripts """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(transcript_ID) FROM observed
                   LEFT JOIN transcript_annotations
                       AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'ISM-suffix_transcript')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    transcripts = [x[0] for x in cursor.fetchall()]
    return transcripts

def fetch_NIC_transcripts(cursor, datasets):
    """ Fetch IDs of all NIC transcripts """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(transcript_ID) FROM observed
                   LEFT JOIN transcript_annotations
                       AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'NIC_transcript')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    transcripts = [x[0] for x in cursor.fetchall()]
    return transcripts

def fetch_NNC_transcripts(cursor, datasets):
    """ Fetch IDs of all NNC transcripts """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(transcript_ID) FROM observed
                   LEFT JOIN transcript_annotations
                       AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'NNC_transcript')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    transcripts = [x[0] for x in cursor.fetchall()]
    return transcripts

def fetch_antisense_transcripts(cursor, datasets):
    """ Fetch IDs of all antisense transcripts """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(transcript_ID) FROM observed
                   LEFT JOIN transcript_annotations
                       AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'antisense_transcript')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    transcripts = [x[0] for x in cursor.fetchall()]
    return transcripts

def fetch_intergenic_transcripts(cursor, datasets):
    """ Fetch IDs of all intergenic transcripts """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(transcript_ID) FROM observed
                   LEFT JOIN transcript_annotations
                       AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'intergenic_transcript')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    transcripts = [x[0] for x in cursor.fetchall()]
    return transcripts


def fetch_genomic_transcripts(cursor, datasets):
    """ Fetch IDs of all genomic transcripts """

    datasets = format_for_IN(datasets)
    query = """SELECT DISTINCT(transcript_ID) FROM observed
                   LEFT JOIN transcript_annotations
                       AS ta ON ta.ID = observed.transcript_ID
                   WHERE (ta.attribute = 'genomic_transcript')
                   AND observed.dataset IN """ + datasets
    cursor.execute(query)
    transcripts = [x[0] for x in cursor.fetchall()]
    return transcripts

def fetch_all_transcript_gene_pairs(cursor):
    """ Return gene_ID - transcript_ID tuples from database """

    query = """ SELECT gene_ID, transcript_ID FROM transcripts """
    cursor.execute(query)
    
    pairs = cursor.fetchall()
    return pairs
    
def fetch_all_datasets(cursor):
    """ Return a list of all datasets in database """
    cursor.execute("SELECT dataset_name FROM dataset")
    datasets = [str(x[0]) for x in cursor.fetchall()]
    return datasets

def parse_whitelist(whitelist_file):
    """ From the whitelist file, obtain a list of acccepted gene and 
        transcript IDs tuples"""
    whitelist = set()
    with open(whitelist_file, 'r') as f:
        for line in f:
            line = line.strip()
            fields = line.split(",")
            gene_ID = fields[0]
            transcript_ID = fields[1]
            try:
                whitelist.add((int(gene_ID), int(transcript_ID)))
            except:
                raise ValueError("Gene/Transcript IDs in whitelist must be integer TALON IDs")
    return whitelist

def parse_datasets(dataset_file, cursor):
    """ From the dataset file, obtain a list of acccepted dataset names"""
    # Get datasets in this database
    db_datasets = fetch_all_datasets(cursor)

    dataset_list = set()
    with open(dataset_file, 'r') as f:
        for line in f:
            line = line.strip()
            fields = line.split()
            dataset = fields[0]
            if dataset not in db_datasets:
                raise ValueError("Dataset name '%s' not found in database" % dataset)
            dataset_list.add(dataset)
    return dataset_list

#-------------------------------------------------------------------------------
def format_for_IN(l):
    """ Converts input to string that can be used for IN database query """
    
    if type(l) is tuple:
        l = list(l)
    if type(l) is str:
        l = [l]

    return "(" + ','.join(['"' + str(x) + '"' for x in l]) + ")" 


def handle_filtering(database, annot, observed, whitelist_file, dataset_file):
    """ Determines which transcripts to allow in the analysis. This can be done
        in two different ways. If no whitelist is included, then all of the
        transcripts in the database are included (modified by 'observed'
        option). If a whitelist is provided, then transcripts on that list
        will be included (modified by 'observed' option). This can be
        tuned further by providing a dataset file, but this is optional. """

    conn = sqlite3.connect(database)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    # Get list of datasets to use in run
    if dataset_file != None:
        datasets = parse_datasets(dataset_file, cursor)
    elif observed == True:
        datasets = fetch_all_datasets(cursor)
    else:
        datasets = None

    # Get initial transcript whitelist
    if whitelist_file != None:
        whitelist = parse_whitelist(whitelist_file)
    else:
        whitelist = fetch_all_transcript_gene_pairs(cursor)

    if datasets != None:
        # Limit the whitelist to transcripts detected in the datasets
        transcripts = [ x[1] for x in whitelist ]
        transcript_str = format_for_IN(transcripts)
        dataset_str = format_for_IN(datasets)

        query = """ SELECT DISTINCT gene_ID, transcript_ID
                    FROM observed
                    WHERE transcript_ID IN %s
                    AND dataset in %s """
        cursor.execute(query % (transcript_str, dataset_str))
        whitelist = cursor.fetchall()

    conn.close()
    return whitelist

def fetch_dataset_list(dataset_file, database):
    """ Gets a list of all datasets in the database """

    conn = sqlite3.connect(database)
    cursor = conn.cursor()
    all_db_datasets = fetch_all_datasets(cursor)
    conn.close()

    if dataset_file == None:

        return all_db_datasets

    else:
        datasets = []
        with open(dataset_file, 'r') as f:
            for line in f:
                dataset = line.strip()
                if dataset not in all_db_datasets:
                    raise ValueError("Dataset name '%s' not found in database" \
                                      % (dataset))
                datasets.append(dataset)

        return datasets

    


In [116]:
def construct_names(gene_ID, transcript_ID, prefix, n_places):
    """ Create a gene and transcript name using the TALON IDs.
        The n_places variable indicates how many characters long the numeric
        part of the name should be. """

    gene_ID_str = str(gene_ID).zfill(n_places)
    gene_name = prefix + "G" + gene_ID_str

    transcript_ID_str = str(transcript_ID).zfill(n_places)
    transcript_name = prefix + "T" + transcript_ID_str

    return gene_name, transcript_name

In [52]:
class Struct(dict):
    """
    Make a dict behave as a struct.

    Example:
    
        test = Struct(a=1, b=2, c=3)

    """
    def __init__(self,**kw):
        dict.__init__(self,kw)
        self.__dict__ = self


In [53]:
# from create_abundance_file that need to go in its own 
# file later jeez

def get_transcript_lengths(database, build):
    """ Read the transcripts from the database. Then compute the lengths.
        Store in a dictionary """

    transcript_lengths = {}

    conn = sqlite3.connect(database)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    # Get the exon lengths
    exon_lens = get_all_exon_lengths(cursor, build)

    cursor.execute("SELECT * FROM transcripts")
    for transcript_row in cursor.fetchall():
        transcript_ID = transcript_row['transcript_ID']
        length = get_transcript_length(transcript_row, exon_lens)
        transcript_lengths[transcript_ID] = length

    conn.close()
    return transcript_lengths

def fetch_naming_prefix(database):
    """ Get naming prefix from the database run_info table """
    conn = sqlite3.connect(database)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    cursor.execute("SELECT value FROM run_info WHERE item = 'idprefix'")
    prefix = cursor.fetchone()[0]

    conn.close()
    return prefix

def fetch_n_places(database):
    """ Get length of name field from the database run_info table """
    conn = sqlite3.connect(database)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    cursor.execute("SELECT value FROM run_info WHERE item = 'n_places'")
    n_places = cursor.fetchone()[0]

    conn.close()
    return int(n_places)

def fetch_dataset_list(dataset_file, database):
    """ Gets a list of all datasets in the database """

    conn = sqlite3.connect(database)
    cursor = conn.cursor()
    all_db_datasets = fetch_all_datasets(cursor)
    conn.close()

    if dataset_file == None:

        return all_db_datasets

    else:
        datasets = []
        with open(dataset_file, 'r') as f:
            for line in f:
                dataset = line.strip()
                if dataset not in all_db_datasets:
                    raise ValueError("Dataset name '%s' not found in database" \
                                      % (dataset))
                datasets.append(dataset)

        return datasets
    
    
def make_novelty_type_struct(database, datasets):
    """ Create a data structure where it is possible to look up whether a gene
        or transcript belongs to a particular category of novelty"""

    conn = sqlite3.connect(database)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    novelty_type = Struct()
    novelty_type.known_genes = set(fetch_all_known_genes_detected(cursor, datasets))
    novelty_type.antisense_genes = set(fetch_antisense_genes(cursor, datasets))
    novelty_type.intergenic_genes = set(fetch_intergenic_novel_genes(cursor, datasets))
    novelty_type.known_transcripts = set(fetch_all_known_transcripts_detected(cursor, datasets))
    novelty_type.ISM_transcripts = set(fetch_all_ISM_transcripts(cursor, datasets))
    novelty_type.ISM_prefix = set(fetch_prefix_ISM_transcripts(cursor, datasets))
    novelty_type.ISM_suffix = set(fetch_suffix_ISM_transcripts(cursor, datasets))
    novelty_type.NIC_transcripts = set(fetch_NIC_transcripts(cursor, datasets))
    novelty_type.NNC_transcripts = set(fetch_NNC_transcripts(cursor, datasets))
    novelty_type.antisense_transcripts = set(fetch_antisense_transcripts(cursor, datasets))
    novelty_type.intergenic_transcripts = set(fetch_intergenic_transcripts(cursor, datasets))
    novelty_type.genomic_transcripts = set(fetch_genomic_transcripts(cursor, datasets))

    conn.close()
    return novelty_type

In [54]:
import pandas as pd
import scanpy as sc
from scipy.sparse import csr_matrix

In [55]:
pass_list = '/Users/fairliereese/Documents/programming/mortazavi_lab/data/mousewg/hippocampus/lr_splitseq/talon/test_pass_list.csv'
db = '/Users/fairliereese/Documents/programming/mortazavi_lab/data/mousewg/hippocampus/lr_splitseq/talon/hippocampus.db'
annot = 'gencode_vM21'
# dataset_file = '/Users/fairliereese/mortazavi_lab/data/mousewg/hippocampus/lr_splitseq/talon/dataset_file.tsv'
dataset_file = None
build = 'mm10'

In [103]:
n_places

9

In [98]:
# transcript_lengths = pd.DataFrame.from_dict(transcript_lengths, orient='index', columns=['length'])

In [74]:
def get_nov_ids_dict(df, d):
    """
    Get a novelty dict for transcripts / genes 
    based on column mapping / values from df
    """
    nov_dict = {}
    for key, val in d.items():
        nov_dict[key] = df.loc[(df.attribute==val[0])&(df.value==val[1]), 'ID'].tolist()
    return nov_dict

In [205]:
def get_g_t_names(db, annot):
    """
    Get names of genes / transcripts
    """

    # get information that we want for each transcript, stuff that 
    # would be output in the abundance table
    query = """
        SELECT
            t.gene_ID,
            t.transcript_ID,
            ga_id.value AS annot_gene_id,
            ta_id.value AS annot_transcript_id,
            ga_name.value AS annot_gene_name,
            ta_name.value AS annot_transcript_name,
            t.n_exons
        FROM transcripts t
            LEFT JOIN gene_annotations ga_id ON t.gene_ID = ga_id.ID
                AND ga_id.annot_name = '%s'
                AND ga_id.attribute = 'gene_id'
            LEFT JOIN transcript_annotations ta_id ON t.transcript_ID = ta_id.ID
                AND ta_id.annot_name = '%s'
                AND ta_id.attribute = 'transcript_id'
            LEFT JOIN gene_annotations ga_name ON t.gene_ID = ga_name.ID
                AND ga_name.annot_name = '%s'
                    AND ga_name.attribute = 'gene_name'
            LEFT JOIN transcript_annotations ta_name ON t.transcript_ID = ta_name.ID
                AND ta_name.annot_name = '%s'
                    AND ta_name.attribute = 'transcript_name'
        """ % (annot, annot, annot, annot)
    df = pd.read_sql_query(query, conn)
    
    return df

In [256]:
def assign_novelties(df, d, order, how):
    
    if how == 'gene':
        nov_col = 'gene_novelty'
        cols = [nov_col]
    elif how == 'transcript':
        nov_col = 'transcript_novelty'
        cols = [nov_col, 'ISM_subtype']
    
    # assign gene or transcript novelty
    df = df.pivot(index='ID', columns=['attribute'], values=['value'])
    df = df.droplevel(0, axis=1)
    df.columns.name = ''

    for key, value in d.items():
        df[key] = False
        df.loc[df[value[0]]==value[1], key] = True
        df.drop(value[0], axis=1, inplace=True)
        
    df[nov_col] = np.nan
    for o in order:
        df.loc[(df[nov_col].isnull())&(df[o]==True), nov_col] = o
    
    # assign ism subtype if needed
    if how == 'transcript':
        df['ISM_subtype'] = np.nan
        df.loc[(df.ISM_subtype.isnull())&(df['ISM-prefix'])&(df['ISM-suffix']), 'ISM_subtype'] = 'Both'
        df.loc[(df.ISM_subtype.isnull())&(df['ISM-prefix']), 'ISM_subtype'] = 'Prefix'
        df.loc[(df.ISM_subtype.isnull())&(df['ISM-suffix']), 'ISM_subtype'] = 'Suffix'
        df.loc[df.ISM_subtype.isnull(), 'ISM_subtype'] = 'None'
    
    # reduce cols
    df = df[cols]
    df.reset_index(inplace=True)
    
    return df

In [257]:
def get_transcript_novs(db):
    nov_col_dict = {'Known': ('transcript_status', 'KNOWN'),
                    'ISM': ('ISM_transcript', 'TRUE'),
                    'ISM-prefix': ('ISM-prefix_transcript', 'TRUE'),
                    'ISM-suffix': ('ISM-suffix_transcript', 'TRUE'),
                    'NIC': ('NIC_transcript', 'TRUE'),
                    'NNC': ('NNC_transcript', 'TRUE'),
                    'Antisense': ('antisense_transcript', 'TRUE'),
                    'Intergenic': ('intergenic_transcript', 'TRUE'),
                    'Genomic': ('genomic_transcript', 'TRUE')}
    order = ['ISM', 'NIC', 'NNC', 'Antisense', 
             'Intergenic', 'Genomic', 'Known']

    attr_list = [val[0] for key, val in nov_col_dict.items()]
    attrs = format_for_IN(attr_list)
    
    with sqlite3.connect(db) as conn:
        query = f"""SELECT ID, attribute, value
                    FROM transcript_annotations 
                    WHERE attribute IN {attrs}
                 """
        df = pd.read_sql_query(query, conn)
        
    df = assign_novelties(df, nov_col_dict, order, 'transcript')

    return df 

In [258]:
def get_gene_novs(db):
    nov_col_dict = {'Known': ('gene_status', 'KNOWN'),
                   'Intergenic': ('intergenic_novel', 'TRUE'),
                   'Antisense': ('antisense_gene', 'TRUE')}
    order = ['Antisense', 'Intergenic', 'Known']
    
    attr_list = [val[0] for key, val in nov_col_dict.items()]
    attrs = format_for_IN(attr_list)
    
    with sqlite3.connect(db) as conn:
        query = f"""SELECT ID, attribute, value
                    FROM gene_annotations 
                    WHERE attribute IN {attrs}
                 """
        df = pd.read_sql_query(query, conn) 
    df = assign_novelties(df, nov_col_dict, order, 'gene')
    
    return df

In [277]:
def get_var_info(db, annot, build):
    
    # get names / ids of transcripts / genes
    df = get_g_t_names(db, annot)
    prefix = fetch_naming_prefix(db)
    n_places = fetch_n_places(db)

    # add gene / transcript names / ids
    df[['temp_gid', 'temp_tid']] = df.apply(lambda x: construct_names(x.gene_ID,
                         x.transcript_ID,
                         prefix,
                         n_places), 
                     axis=1, result_type='expand')

    # replace null gene names / ids
    inds = df.loc[df.annot_gene_id.isnull()].index
    df.loc[inds, 'annot_gene_id'] = df.loc[inds, 'temp_gid']
    inds = df.loc[df.annot_gene_name.isnull()].index
    df.loc[inds, 'annot_gene_name'] = df.loc[inds, 'temp_gid']

    # replace null transcript names / ids
    inds = df.loc[df.annot_transcript_id.isnull()].index
    df.loc[inds, 'annot_transcript_id'] = df.loc[inds, 'temp_tid']
    inds = df.loc[df.annot_transcript_name.isnull()].index
    df.loc[inds, 'annot_transcript_name'] = df.loc[inds, 'temp_tid']

    # remove temp cols
    df.drop(['temp_gid', 'temp_tid'], axis=1, inplace=True)
    
    # add transcript len
    t_lens = pd.DataFrame.from_dict(get_transcript_lengths(db, build),
                                                orient='index',
                                                columns=['length'])
    df = df.merge(t_lens, how='left', left_on='transcript_ID', right_index=True)
    
    # add gene novelty
    g_df = get_gene_novs(db)
    df = df.merge(g_df, how='left', left_on='gene_ID', right_on='ID')
    df.drop('ID', axis=1, inplace=True)

    # add transcript novelty / ism subtype
    t_df = get_transcript_novs(db)
    df = df.merge(t_df, how='left', left_on='transcript_ID', right_on='ID')
    df.drop('ID', axis=1, inplace=True)
    
    # column order 
    order = ['gene_ID', 'transcript_ID', 'annot_gene_id', 
             'annot_transcript_id', 'annot_gene_name',
             'annot_transcript_name', 'n_exons', 'length',
             'gene_novelty', 'transcript_novelty', 'ISM_subtype']
    df = df[order]
    
    return df


In [278]:
df = get_var_info(db, annot, build)

In [263]:
# get names / ids of transcripts / genes
df = get_g_t_names(db, annot)
prefix = fetch_naming_prefix(db)
n_places = fetch_n_places(db)

# add gene / transcript names / ids
df[['temp_gid', 'temp_tid']] = df.apply(lambda x: construct_names(x.gene_ID,
                     x.transcript_ID,
                     prefix,
                     n_places), 
                 axis=1, result_type='expand')

# replace null gene names / ids
inds = df.loc[df.annot_gene_id.isnull()].index
df.loc[inds, 'annot_gene_id'] = df.loc[inds, 'temp_gid']
inds = df.loc[df.annot_gene_name.isnull()].index
df.loc[inds, 'annot_gene_name'] = df.loc[inds, 'temp_gid']

# replace null transcript names / ids
inds = df.loc[df.annot_transcript_id.isnull()].index
df.loc[inds, 'annot_transcript_id'] = df.loc[inds, 'temp_tid']
inds = df.loc[df.annot_transcript_name.isnull()].index
df.loc[inds, 'annot_transcript_name'] = df.loc[inds, 'temp_tid']

# remove temp cols
df.drop(['temp_gid', 'temp_tid'], axis=1, inplace=True)

In [264]:
# add transcript len
t_lens = pd.DataFrame.from_dict(get_transcript_lengths(db, build),
                                            orient='index',
                                            columns=['length'])
df = df.merge(t_lens, how='left', left_on='transcript_ID', right_index=True)

In [265]:
df.head()

,gene_ID,transcript_ID,annot_gene_id,annot_transcript_id,annot_gene_name,annot_transcript_name,n_exons,length
0,1,1,ENSMUSG00000102693.1,ENSMUST00000193812.1,4933401J01Rik,4933401J01Rik-201,1,1070
1,2,2,ENSMUSG00000064842.1,ENSMUST00000082908.1,Gm26206,Gm26206-201,1,110
2,3,3,ENSMUSG00000051951.5,ENSMUST00000162897.1,Xkr4,Xkr4-203,2,4153
3,3,4,ENSMUSG00000051951.5,ENSMUST00000159265.1,Xkr4,Xkr4-202,2,2989
4,3,5,ENSMUSG00000051951.5,ENSMUST00000070533.4,Xkr4,Xkr4-201,3,3634


In [274]:
# add gene novelty
g_df = get_gene_novs(db)
df = df.merge(g_df, how='left', left_on='gene_ID', right_on='ID')
df.drop('ID', axis=1, inplace=True)

# add transcript novelty / ism subtype
t_df = get_transcript_novs(db)
df = df.merge(t_df, how='left', left_on='transcript_ID', right_on='ID')
df.drop('ID', axis=1, inplace=True)


In [272]:
print(max(df.gene_ID))
print(max(g_df.ID))
print(min(df.gene_ID))
print(min(g_df.ID))
g_df.head()

636549
636549
1
1


,ID,gene_novelty
0,1,Known
1,2,Known
2,3,Known
3,4,Known
4,5,Known


In [273]:
print(max(df.transcript_ID))
print(max(t_df.ID))
print(min(df.transcript_ID))
print(min(t_df.ID))
t_df.head()

2459184
2459184
1
1


,ID,transcript_novelty,ISM_subtype
0,1,Known,None
1,2,Known,None
2,3,Known,None
3,4,Known,None
4,5,Known,None


In [114]:
# df.loc[df.length.isnull()]

In [115]:
df.loc[df.annot_gene_id.isnull()].head()

,gene_ID,transcript_ID,annot_gene_id,annot_transcript_id,annot_gene_name,annot_transcript_name,n_exons,length
141862,55537,141863,None,None,None,None,1,957
141863,55538,141864,None,None,None,None,2,290
141865,55538,141866,None,None,None,None,1,543
141866,55539,141867,None,None,None,None,1,1182
141867,55540,141868,None,None,None,None,1,206


In [88]:
# get novelty types of genes
g_df, g_dict = get_gene_novs(db)

# get novelty types of transcripts
t_df, t_dict = get_transcript_novs(db)

In [23]:
# whitelist = putils.handle_filtering(db, 
#                             annot, 
#                             False, 
#                             pass_list, 
#                             None)
whitelist = handle_filtering(db, 
                            annot, 
                            False, 
                            pass_list, 
                            dataset_file)

datasets = fetch_dataset_list(dataset_file, db)


In [24]:
transcripts = [ x[1] for x in whitelist ]
transcript_str = format_for_IN(transcripts)
dataset_str = format_for_IN(datasets)

In [25]:
# X data
with sqlite3.connect(db) as conn:
    query = f"""SELECT transcript_ID, dataset, count
                FROM abundance WHERE transcript_ID in {transcript_str}
                AND dataset in {dataset_str}
             """
    df = pd.read_sql_query(query, conn)

In [26]:
# df.pivot(index='dataset', columns='transcript_ID', values='count')

In [27]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

obs_col = 'dataset'
var_col = 'transcript_ID'

obs_cat = CategoricalDtype(sorted(df[obs_col].unique()), ordered=True)
var_cat = CategoricalDtype(sorted(df[var_col].unique()), ordered=True)

row = df[obs_col].astype(obs_cat).cat.codes
col = df[var_col].astype(var_cat).cat.codes
X = csr_matrix((df['count'], (row, col)), \
               shape=(obs_cat.categories.size,
                      var_cat.categories.size))


In [28]:
X

<9561x2 sparse matrix of type '<class 'numpy.int64'>'
	with 9590 stored elements in Compressed Sparse Row format>

In [29]:
df = pd.DataFrame.sparse.from_spmatrix(X, \
                         index=obs_cat.categories, \
                         columns=var_cat.categories)

In [30]:
df

,141732,152466
AAACATCGAAACATCGTCACTTTA-ont_2ka,1,0
AAACATCGAACAACCAATTCATGG-ont_2ka,1,0
AAACATCGAACAACCAGACCTTTC-pb,1,0
AAACATCGAACAACCAGCTCGCGG-ont_2ka,4,0
AAACATCGAACCGAGATCACTTTA-ont_2ka,1,0
...,...,...
TTCACGCATCTTCACACGTTCGAG-ont_2kb,1,0
TTCACGCATCTTCACAGCTCGCGG-ont_2kb,7,0
TTCACGCATCTTCACATCACTTTA-ont_2kb,1,0
TTCACGCATGAAGAGAACTATATA-ont_2ka,1,0


In [31]:
# obs
obs = pd.DataFrame(data=obs_cat.categories.tolist(), columns=['dataset'])
obs.head()

# get information

,dataset
0,AAACATCGAAACATCGTCACTTTA-ont_2ka
1,AAACATCGAACAACCAATTCATGG-ont_2ka
2,AAACATCGAACAACCAGACCTTTC-pb
3,AAACATCGAACAACCAGCTCGCGG-ont_2ka
4,AAACATCGAACCGAGATCACTTTA-ont_2ka


In [34]:
# var
var = pd.DataFrame(data=var_cat.categories.tolist(), columns=['transcript_ID'])
var.head()





In [35]:
df.loc[~df.annot_gene_id.isnull()]

,gene_ID,transcript_ID,annot_gene_id,annot_transcript_id,annot_gene_name,annot_transcript_name,n_exons
0,1,1,ENSMUSG00000102693.1,ENSMUST00000193812.1,4933401J01Rik,4933401J01Rik-201,1
1,2,2,ENSMUSG00000064842.1,ENSMUST00000082908.1,Gm26206,Gm26206-201,1
2,3,3,ENSMUSG00000051951.5,ENSMUST00000162897.1,Xkr4,Xkr4-203,2
3,3,4,ENSMUSG00000051951.5,ENSMUST00000159265.1,Xkr4,Xkr4-202,2
4,3,5,ENSMUSG00000051951.5,ENSMUST00000070533.4,Xkr4,Xkr4-201,3
...,...,...,...,...,...,...,...
2459168,7245,2459169,ENSMUSG00000027589.14,None,Pcmtd2,None,4
2459170,7245,2459171,ENSMUSG00000027589.14,None,Pcmtd2,None,1
2459171,7245,2459172,ENSMUSG00000027589.14,None,Pcmtd2,None,1
2459174,7246,2459175,ENSMUSG00000038628.8,None,Polr3k,None,2
